In [1]:
import jax
import jax.numpy as jnp
from specq_jax.pulse import DragPulse, JaxBasedPulseSequence
from specq_jax.core import get_simulator, calculate_expvals, plot_expvals
import specq_dev.shared as specq
from specq_dev.qiskit import get_circuit, get_ibm_service_and_backend, IBMQDeviceProperties, transpile_circuits
from qiskit import schedule
import matplotlib.pyplot as plt

from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_ibm_runtime import Session, Options, Estimator #SamplerV2 as Sampler
from qiskit import QuantumCircuit
from alive_progress import alive_it
import pandas as pd
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.primitives import EstimatorResult

In [2]:

def define_config_and_pulse_sequence():

    instance = "utokyo-kawasaki/keio-internal/keio-students"
    backend_name = "ibm_kawasaki"

    service, backend = get_ibm_service_and_backend(instance, backend_name)

    device_properties = IBMQDeviceProperties.from_backend(backend, qubit_indices=[0])

    qubit_info = device_properties.qubit_informations[0]

    dt = device_properties.dt

    total_length = 320

    amp = 0.25  # NOTE: Choice of amplitude is arbitrary
    area = (
        1 / (2 * qubit_info.drive_strength) / dt
    )  # NOTE: Choice of area is arbitrary e.g. pi pulse
    sigma = (1 * area) / (amp * jnp.sqrt(2 * jnp.pi))

    pulse_sequence = JaxBasedPulseSequence(
        pulses=[
            DragPulse(
                total_length=total_length,
                base_amp=amp,
                base_sigma=float(sigma),
                base_beta=1 / qubit_info.anharmonicity,
                total_amps=1,
                max_amp=0,
                min_amp=0,
                final_amp=1.0,
            )
        ],
        pulse_length_dt=total_length,
    )

    t_eval = jax.numpy.linspace(0, total_length * dt, 100)

    simulator = get_simulator(qubit_info=qubit_info, t_eval=t_eval)

    measurement_timings = jnp.arange(
        device_properties.min_length, total_length + 1, device_properties.granularity
    )

    key = jax.random.PRNGKey(0)
    params = pulse_sequence.sample_params(key)
    waveform = pulse_sequence.get_waveform(params)
    waveforms = [waveform[:t] for t in measurement_timings]

    exp_config = specq.ExperimentConfigV3(
        qubits=[qubit_info],
        expectation_values_order=specq.default_expectation_values,
        parameter_names=[],
        backend_name=backend.name,
        shots=3000,
        EXPERIMENT_IDENTIFIER="whitebox_test_0001",
        EXPERIMENT_TAGS=["whitebox_test", "DRAG"],
        description="Whitebox test with DRAG pulse",
        device_cycle_time_ns=device_properties.dt,
        sequence_duration_dt=total_length,
        instance=instance,
        sample_size=18 * len(measurement_timings),
        additional_info=dict(
            pulse_class="DragPulse",
            num_pulses=1,
        ),
    )

    return exp_config, pulse_sequence, service, backend, simulator, waveforms

def check_job_status(data: pd.DataFrame, service: QiskitRuntimeService):

    # Assume that data has a column named "job_id"
    # Get unqiue job ids
    job_ids = data["job_id"].unique()

    status = []
    # Get the status of the jobs for each job id
    for job_id in job_ids:
        job = service.job(job_id)
        print(f"Job {job_id} status: {job.status()}")
        status.append(job.status())

    return status

def plot_result(
        waveforms, 
        results_df,
        exp_config,
        simulator,
    ):

    unitaries = simulator(waveforms[-1])
    expvals = calculate_expvals(unitaries)

    fig, axes = plt.subplots(3, 6, sharex=True, sharey=True, figsize=(18, 9))

    for idx, exp in enumerate(specq.default_expectation_values):
        rows = results_df.loc[
            (results_df["initial_state"] == exp.initial_state)
            & (results_df["observable"] == exp.observable)
        ][["duration", "expectation_value", "global_phase"]]

        values = rows.values

        t = values[:, 0] * exp_config.device_cycle_time_ns
        y = values[:, 1]

        global_phase = rows["global_phase"].unique()

        dt = exp_config.device_cycle_time_ns
        total_length = exp_config.sequence_duration_dt

        y_simulator = expvals[idx]
        t_eval_simulator = jax.numpy.linspace(0, total_length * dt, 100)

        ax = axes[idx // 6, idx % 6]
        ax.plot(t, y, "o", label="IBM")
        ax.plot(t_eval_simulator, y_simulator, label="Simulator")

        assert global_phase.shape == (1, )

        ax.set_title(f"{exp.initial_state} -> {exp.observable}: \n global phase = {(round(global_phase[0], 5))}")

        ax.legend()
    return fig

In [3]:
def get_circuits_and_data(
    backend,
    exp_config,
    waveforms,
    use_sampler=True
):

    data = []
    circuits = []
    idx = 0

    for exp in specq.default_expectation_values:
        for waveform in waveforms:

            metadata = dict(
                duration=waveform.shape[0],
                initial_state=exp.initial_state,
                observable=exp.observable,
                idx=idx,
            )

            circuit = get_circuit(
                initial_state=exp.initial_state,
                waveforms=waveform,
                observable=exp.observable,
                q_idx=exp_config.qubits[0].qubit_idx,
                backend=backend,
                add_pulse=True,
                change_basis=use_sampler,
                add_measure=use_sampler,
                enable_MCMD=use_sampler,
                metadata=metadata,
            )
            circuits.append(circuit)
            data.append(
                dict(
                    job_id=None,
                    **metadata,
                )
            )
            idx += 1

    return circuits, data

In [4]:
def run_experiment_with_estimator(
    quantum_circuits: list[QuantumCircuit],
    backend,
    exp_config: specq.ExperimentConfigV3,
    data: pd.DataFrame,
):

    max_experiment_per_job_in_runtime = backend.configuration().max_experiments
    total_experiments = len(quantum_circuits)
    total_jobs = total_experiments // max_experiment_per_job_in_runtime
    
    if total_experiments % max_experiment_per_job_in_runtime != 0:
        total_jobs += 1

    options = Options()
    
    OPTIMIZATION_LEVEL = 0
    RESILIENCE_LEVEL = 0
    SKIP_TRANSPILE = True
    initial_layout = [exp_config.qubits[0].qubit_idx]

    options.execution.shots = exp_config.shots
    options.optimization_level = OPTIMIZATION_LEVEL
    options.resilience_level = RESILIENCE_LEVEL
    options.environment.job_tags = exp_config.EXPERIMENT_TAGS
    options.transpilation.skip_transpilation = SKIP_TRANSPILE
    options.transpilation.initial_layout = initial_layout

    with Session(backend=backend) as session:
        estimator = Estimator(session=session, options=options)

        for job_idx in alive_it(
            range(total_jobs),
            bar="smooth",
            spinner="dots_waves",
            force_tty=True,
        ):
            # Get the start and end index of the job_idx-th job
            start_idx = int(job_idx * max_experiment_per_job_in_runtime)
            end_idx = int((job_idx + 1) * max_experiment_per_job_in_runtime)

            # Get quantum circuits
            qcs_list = quantum_circuits[start_idx:end_idx]

            observables = []
            for qc in qcs_list:
                ob = qc.metadata['observable']
                concat_ob = ["I" * (backend.num_qubits - 1) + ob]
                observables.append(concat_ob)

            # Run the job
            job = estimator.run(circuits=qcs_list, observables=observables)

            # Get the job id and set the job id ti the job_idx-th job
            job_id = job.job_id()

            # data.loc[start_idx:end_idx, "job_idx"] = job_idx
            data.loc[start_idx:end_idx, "job_id"] = job_id
            data.loc[start_idx:end_idx, "job_idx"] = job_idx

    return data

def get_estimator_results_from_remote(
    service: QiskitRuntimeService,
    backend,
    data: pd.DataFrame,
):

    max_experiment_per_job_in_runtime = backend.configuration().max_experiments
    total_experiments = len(data)
    total_jobs = total_experiments // max_experiment_per_job_in_runtime

    if total_experiments % max_experiment_per_job_in_runtime != 0:
        total_jobs += 1

    for job_idx in alive_it(
        range(total_jobs),
        bar="smooth",
        spinner="dots_waves",
        force_tty=True,
    ):
        # NOTE: Confidently assume that job_id is unique for each job index
        job_id = data.loc[data["job_idx"] == job_idx, ("job_id")].unique()[0]

        # Get the result of the job
        job = service.job(job_id=job_id)

        success = job.done()

        if success:
            result: EstimatorResult = job.result()

            data.loc[data["job_idx"] == job_idx, ("expectation_value")] = result.values
            data.loc[data["job_idx"] == job_idx, ("status")] = "done"

        else:
            data.loc[data["job_idx"] == job_idx, ("status")] = "failed"

    return data

In [9]:
PATH = "../tests/test_data/drag_0002/"
READ_MODE = False

exp_config, pulse_sequence, service, backend, simulator, waveforms = (
    define_config_and_pulse_sequence()
)

circuits, data = get_circuits_and_data(
    backend=backend,
    exp_config=exp_config,
    waveforms=waveforms,
    use_sampler=False,
)
trans_qcs = transpile_circuits(
    circuits, backend=backend, inital_layout=[exp_config.qubits[0].qubit_idx]
)

if not READ_MODE:
    # Execute the experiment 
    data = pd.DataFrame(data)
    df = run_experiment_with_estimator(trans_qcs, backend, exp_config, data)

    exp_config.to_file(PATH)
    pulse_sequence.to_file(PATH)
    df.to_csv(PATH + "data.csv", index=False)

    status = check_job_status(df, service)

    results_df = get_estimator_results_from_remote(service, backend, df)
else:
    results_df = pd.read_csv(PATH + "data.csv")

global_phases = []
for qc in trans_qcs:
    global_phases.append(qc.global_phase)

results_df["global_phase"] = global_phases

results_df.to_csv(PATH + "data.csv", index=False)

/var/folders/rm/_5r9_x2s3tgfyqlnv1pdjfqc0000gn/T/ipykernel_78154/629217803.py:30: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  estimator = Estimator(session=session, options=options)


|████████████████████████████████████████| 2/2 [100%] in 5.4s (0.27/s)          
Job crh4mg269sp0008qhhfg status: JobStatus.RUNNING
Job crh4mgjqzd50008881h0 status: JobStatus.RUNNING
|████████████████████████████████████████| 2/2 [100%] in 1.3s (1.29/s)          


In [10]:
results_df = get_estimator_results_from_remote(service, backend, df)

|████████████████████████████████████████| 2/2 [100%] in 1:08.9 (0.02/s)        


In [11]:
results_df.to_csv(PATH + "data.csv", index=False)

In [12]:
results_df

,job_id,duration,initial_state,observable,idx,job_idx,status,global_phase,expectation_value
0,crh4mg269sp0008qhhfg,64,+,X,0,0.0,done,0.785398,0.978667
1,crh4mg269sp0008qhhfg,80,+,X,1,0.0,done,0.785398,0.982000
2,crh4mg269sp0008qhhfg,96,+,X,2,0.0,done,0.785398,0.980667
3,crh4mg269sp0008qhhfg,112,+,X,3,0.0,done,0.785398,0.972000
4,crh4mg269sp0008qhhfg,128,+,X,4,0.0,done,0.785398,0.968667
...,...,...,...,...,...,...,...,...,...
301,crh4mgjqzd50008881h0,256,1,Z,301,1.0,done,0.000000,0.972667
302,crh4mgjqzd50008881h0,272,1,Z,302,1.0,done,0.000000,0.972667
303,crh4mgjqzd50008881h0,288,1,Z,303,1.0,done,0.000000,0.968000
304,crh4mgjqzd50008881h0,304,1,Z,304,1.0,done,0.000000,0.964000
